In [ ]:
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from torchvision import transforms
import sys
import os
sys.path.append(os.path.abspath(".."))
from models import model_dict
from utils import NormalizeByChannelMeanStd
import numpy as np
from torchvision.datasets import CIFAR10, CIFAR100, ImageFolder
from dataset import prepare_train_test_dataset
from torch.utils.data import DataLoader, Dataset, Subset
import torch
import pickle
from itertools import cycle
from utils.evaluation import Hook_handle, analysis, get_micro_eval, get_acc, get_micro_eval_seperate_correct
import pandas as pd
import argparse
import random
import copy
from types import SimpleNamespace
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


dataset = ['cifar10', 'cifar100', 'TinyImagenet', 'TinyImagenet'] # 
architecture = ['resnet18', 'resnet50', 'resnet18', 'vgg16_bn']



target = 'forget'

for method in ["randomlabel_salun", "randomlabel"]:
    for arch, data in zip(architecture, dataset):
        for us in range(1):
            train_loader, test_loader, forget_loader, retain_loader, normalization, classes = prepare_train_test_dataset(data, 0, 256, us)
            with open(f"assets/feature_topo_analysis/{data}_{arch}_unlearn_seed_{us}_feature_label.pkl", "rb") as f:
                backdata = pickle.load(f)
            with open(f"assets/unlearn_feature_topo_analysis/{method}_{data}_{arch}_unlearn_seed_{us}_feature_label.pkl", "rb") as f:
                backdata_unlearn = pickle.load(f)
            for init_seed in range(1):
                orig_feature = backdata[init_seed]['orig_feature'][target]
                retrain_feature = backdata[init_seed]['retrain_feature'][target]
                standard_feature = backdata_unlearn[init_seed]['standard_feature'][target]
                periun_feature = backdata_unlearn[init_seed]['periun_feature'][target]

                orig_weight = backdata[init_seed]['orig_weight'][target]
                retrain_weight = backdata[init_seed]['retrain_weight'][target]
                standard_weight = backdata_unlearn[init_seed]['standard_weight'][target]
                periun_weight = backdata_unlearn[init_seed]['periun_weight'][target]

                orig_bias = backdata[init_seed]['orig_bias'][target]
                retrain_bias = backdata[init_seed]['retrain_bias'][target]
                standard_bias = backdata_unlearn[init_seed]['standard_bias'][target]
                periun_bias = backdata_unlearn[init_seed]['periun_bias'][target]

                orig_pred = torch.max(orig_feature @ orig_weight.T + orig_bias, dim=1).indices
                retrain_pred = torch.max(retrain_feature @ retrain_weight.T + retrain_bias, dim=1).indices
                standard_pred = torch.max(standard_feature @ standard_weight.T + standard_bias, dim=1).indices
                periun_pred = torch.max(periun_feature @ periun_weight.T + periun_bias, dim=1).indices
                
                label = backdata[init_seed]['retrain_label'][target]

                orig_correct = (orig_pred == label)
                retrain_correct = (retrain_pred == label)
                retrain_fliped = (~retrain_correct) & orig_correct

                standard_correct = (standard_pred == label)
                standard_fliped = (~standard_correct) & orig_correct
                periun_correct = (periun_pred == label)
                periun_fliped = (~periun_correct) & orig_correct

                retrain_standard_agreement = torch.sum(retrain_fliped == standard_fliped).item() / retrain_correct.shape[0]
                retrain_periun_agreement = torch.sum(retrain_fliped == periun_fliped).item() / retrain_correct.shape[0]
                print(f"fliped Agreement {method} {data} {arch} {us} {init_seed}: retrain_standard_agreement: {retrain_standard_agreement:.2f}, retrain_periun_agreement: {retrain_periun_agreement:.2f}")
            
                orig_conf = backdata[init_seed]['orig_confidence'][target]
                retrain_conf = backdata[init_seed]['retrain_confidence'][target]
                standard_conf = backdata_unlearn[init_seed]['standard_confidence'][target]
                periun_conf = backdata_unlearn[init_seed]['periun_confidence'][target]

                retrain_fliped_conf = orig_conf[retrain_fliped]
                standard_fliped_conf = orig_conf[standard_fliped]
                periun_fliped_conf = orig_conf[periun_fliped]

                top10_indices = orig_conf.argsort(descending=True)[:10]

                images = [forget_loader.dataset[idx][0].permute(1, 2, 0) for idx in top10_indices]

                # titles = [f'Top-{i+1}' for i in range(15)]

                fig, axes = plt.subplots(2, 5, figsize=(5.5, 2))  
                for ax, img in zip(axes.flatten(), images):
                    ax.imshow(img, vmin=0, vmax=1)
                    # ax.set_title(title)
                    ax.axis('off')

                plt.tight_layout()
                plt.savefig(f'assets/final_quantative/{method}_{data}_{arch}_{us}_{init_seed}_vispic.pdf', bbox_inches='tight', format='pdf')
                plt.show()

                top15_indices = orig_conf.argsort(descending=True)[:15]

                flags = {
                    'Retrain': retrain_fliped[top15_indices].tolist(),
                    'Standard': standard_fliped[top15_indices].tolist(),
                    'Periun': periun_fliped[top15_indices].tolist()
                }
                # flags = {
                #     'Retrain': ~retrain_correct[top15_indices].tolist(),
                #     'Standard': ~standard_correct[top15_indices].tolist(),
                #     'Periun': ~periun_correct[top15_indices].tolist()
                # }



                cell_width = 0.35
                cell_height = 0.4
                h_gap = 0.1
                v_gap = 0.2

                n_cols = len(next(iter(flags.values())))
                n_rows = len(flags)

                fig_width = n_cols * (cell_width + h_gap)
                fig_height = n_rows * (cell_height + v_gap) - v_gap

                fig, ax = plt.subplots(figsize=(fig_width, fig_height))

                row_labels = [r'$\mathcal{M}_r$', r'$\mathcal{M}_\text{base}$', r'$\mathcal{M}_\text{ours}$']


                for row_idx, (label, values) in enumerate(flags.items()):
                    for col_idx, val in enumerate(values):
                        x = col_idx * (cell_width + h_gap)
                        y = -row_idx * (cell_height + v_gap)
                        color = 'red' if val else 'green'
                        rect = Rectangle((x, y), cell_width, cell_height, facecolor=color, edgecolor='black')
                        ax.add_patch(rect)
     
                    ax.text(-0.2, y + cell_height / 2, row_labels[row_idx], va='center', ha='right', fontsize=25)

                
                x_max = n_cols * (cell_width + h_gap) - h_gap
                y_min = -n_rows * (cell_height + v_gap) + v_gap + v_gap + 0.175

                ax.set_xlim(-1, x_max) 
                ax.set_ylim(y_min, cell_height)
                ax.set_aspect('equal')
                ax.axis('off')

                plt.savefig(f'assets/final_quantative/{method}_{data}_{arch}_{us}_{init_seed}_filped.pdf', bbox_inches='tight', format='pdf')
                plt.show()
            
            del backdata, backdata_unlearn
